In [10]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False)

def extract_features(directory, model, target_size=(150, 150)):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=1,
        class_mode=None,
        shuffle=False
    )
    num_images = len(generator.filenames)
    features = np.zeros(shape=(num_images, 4 * 4 * 512))
    for i, img_path in enumerate(generator.filepaths):
        img = load_img(img_path, target_size=target_size)
        img_data = img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        features[i] = model.predict(img_data).flatten()
    return features, generator.classes


train_dir = r"C:\Users\Idriss\Desktop\PFA final sumbmission\archive(2)\train"
validation_dir = r"C:\Users\Idriss\Desktop\PFA final sumbmission\archive(2)\test"

train_features, train_labels = extract_features_from_directory(train_dir, vgg_model)
validation_features, validation_labels = extract_features_from_directory(validation_dir, vgg_model)


Found 557 images belonging to 2 classes.
1/1 [==============================] - 0s 52ms/step


1/1 [==============================] - 0s 57ms/step


1/1 [==============================] - 0s 54ms/step


1/1 [==============================] - 0s 60ms/step
Found 140 images belonging to 2 classes.
1/1 [==============================] - 0s 53ms/step


1/1 [==============================] - 0s 56ms/step


In [11]:
def gale_shapley(source_features, target_features):
    n = len(source_features)
    free_sources = list(range(n))
    free_targets = list(range(n))
    matches = [-1] * n
    target_preferences = [list(range(n)) for _ in range(n)]
    
    while free_sources:
        source = free_sources.pop(0)
        target = target_preferences[source].pop(0)
        
        if target in free_targets:
            matches[source] = target
            free_targets.remove(target)
        else:
            current_source = matches.index(target)
            if compare_features(source_features[source], target_features[target]) > compare_features(source_features[current_source], target_features[target]):
                matches[current_source] = -1
                free_sources.append(current_source)
                matches[source] = target
    return matches

def compare_features(source, target):
    return np.dot(source, target)  # Example similarity measure (dot product)

matches = gale_shapley(train_features, validation_features)
print(matches)


[4, 3, 2, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 

In [12]:
# Create new training and validation datasets
matched_train_features = np.zeros_like(train_features)
matched_validation_features = np.zeros_like(validation_features)

for i, match in enumerate(matches):
    if match != -1:
        matched_train_features[i] = train_features[i]
        matched_validation_features[i] = validation_features[match]

matched_train_labels = train_labels
matched_validation_labels = validation_labels[matches]


In [17]:
matched_train_labels.shape


(557,)

In [18]:
matched_validation_labels.shape

(557,)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model


model = load_model('dog_cat_classifier_vgg16.h5')

# Train the model using the matched features
history = model.fit(
    matched_train_features,
    matched_train_labels, 
    epochs=20,
    batch_size=32,
    validation_data=(matched_validation_features, matched_validation_labels)
)

# Evaluate the model
loss, accuracy = model.evaluate(matched_validation_features, matched_validation_labels)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')



ValueError: cannot reshape array of size 4562944 into shape (150,150,3)